In [4]:
import networkx as nx

# Path to your local edge-list file
path = "web-Google.txt"

# Read into a directed graph
G = nx.read_edgelist(
    path,
    comments="#",            # ignore SNAP header lines
    delimiter="\t",          # tab-separated
    create_using=nx.DiGraph(),  # build a DiGraph
    nodetype=int             # cast node IDs to int
)

In [6]:
# Print basic info manually
print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")

Nodes: 875713
Edges: 5105039


In [7]:
# Compute PageRank
pr = nx.pagerank(G, alpha=0.85, tol=1e-6, max_iter=100)

# Example: print the top 5 nodes by PageRank
top5 = sorted(pr.items(), key=lambda x: x[1], reverse=True)[:5]
for node, score in top5:
    print(f"Node {node}: {score:.6f}")

Node 163075: 0.000952
Node 597621: 0.000901
Node 537039: 0.000895
Node 837478: 0.000876
Node 885605: 0.000822
